In [ ]:
!pip install lightly

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 660.5/660.5 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 96.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 727.0/727.0 kB 65.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 17.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 763.4/763.4 kB 67.9 MB/s eta 0:00:00
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.9.3-py3-none-any.whl size=144554 sha256=94d9e502bcb7227a6336e95277d9ae6f3c9ee0933d59e3a600e6316b768d58dc
  Stored in directory: /root/.cache/pip/wheels/12/93/dd/1f6a127edc45659556564c5730f6d4e300888f4bca2d4c5a88
Successfully built antl

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip '/content/drive/MyDrive/Kaggle/data.zip' -d '/content/'

Streaming output truncated to the last 5000 lines.
  inflating: /content/images_012/images/00028897_016.png  
  inflating: /content/images_012/images/00028897_017.png  
  inflating: /content/images_012/images/00028897_018.png  
  inflating: /content/images_012/images/00028897_019.png  
  inflating: /content/images_012/images/00028897_020.png  
  inflating: /content/images_012/images/00028898_000.png  
  inflating: /content/images_012/images/00028899_000.png  
  inflating: /content/images_012/images/00028899_001.png  
  inflating: /content/images_012/images/00028899_002.png  
  inflating: /content/images_012/images/00028900_000.png  
  inflating: /content/images_012/images/00028901_000.png  
  inflating: /content/images_012/images/00028901_001.png  
  inflating: /content/images_012/images/00028902_000.png  
  inflating: /content/images_012/images/00028902_001.png  
  inflating: /content/images_012/images/00028902_002.png  
  inflating: /content/images_012/images/00028902_003.png  
  inf

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from cv2 import imread, createCLAHE # read and equalize images
from glob import glob
import matplotlib.pyplot as plt
%matplotlib inline
from pandas.core.common import flatten

import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

plt.ion()   # interactive mode

In [ ]:
from __future__ import print_function, division

# pytorch imports
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

# image imports
from skimage import io, transform
from PIL import Image

# general imports
import os
import time
from shutil import copyfile
from shutil import rmtree

# data science imports
import pandas as pd
import numpy as np
import csv

In [ ]:
from itertools import chain
import random
np.random.seed(25)

df = pd.read_csv('/content/Data_Entry_2017.csv')
df.drop(['OriginalImage[Width', 'Height]', 'OriginalImagePixelSpacing[x', 'y]', 'Unnamed: 11'], axis=1, inplace=True)
num_obs = len(df)
# print('Number of observations:',num_obs)

my_glob = glob('/content/images*/images/*.png')
# print('Number of Observations: ', len(my_glob))

full_img_paths = {os.path.basename(x): x for x in my_glob}
df['full_path'] = df['Image Index'].map(full_img_paths.get)

train_val_list = pd.read_csv('/content/train_val_list.txt', header=None, names = ['image_list'])
test_list = pd.read_csv('/content/test_list.txt', header=None, names = ['image_list'])

train = df[df['Image Index'].isin(train_val_list['image_list'].values)].reset_index(drop=True)
test = df[df['Image Index'].isin(test_list['image_list'].values)].reset_index(drop=True)

labels_discard = ['Consolidation', 'Edema', 'Emphysema', 'Fibrosis', 'Hernia', 'Pleural_Thickening']
for i in labels_discard:
    train = train[~train['Finding Labels'].str.contains(i)]
    test = test[~test['Finding Labels'].str.contains(i)]

# train = pd.concat([train[~train['Finding Labels'].str.contains('No Finding')],
#                   train[train['Finding Labels'].str.contains('No Finding')].drop_duplicates(subset=['Finding Labels', 'Patient ID', 'View Position'], keep='first')]).sort_values(by='Image Index')

def one_hot_enc(df):
    df['Finding Labels'] = df['Finding Labels'].map(lambda x: x.replace('No Finding', ''))
    all_labels = np.unique(list(chain(*df['Finding Labels'].map(lambda x: x.split('|')).tolist())))
    for c_label in all_labels:
        if len(c_label)>1: # leave out empty labels
            df[c_label] = df['Finding Labels'].map(lambda finding: 1 if c_label in finding else 0)
    return df

train = one_hot_enc(train)
test = one_hot_enc(test)

train_image_paths = list(flatten(train['full_path'].values))
test_image_paths = list(flatten(test['full_path'].values))

l = np.unique(train['Patient ID'].values)
np.random.shuffle(l)
cut = int(np.round(((90/100)*len(l)), decimals=0))
train_values = l[:cut]
val_values = l[cut:]
train_dict = dict.fromkeys(train_values, 'train')
train_dict.update(dict.fromkeys(val_values, 'val'))
train['fold'] = train['Patient ID'].map(train_dict.get)

train = train[['full_path', 'Image Index', 'fold', 'Atelectasis', 'Cardiomegaly', 'Effusion', 'Infiltration',
               'Mass', 'Nodule', 'Pneumonia', 'Pneumothorax']]
test['fold'] = 'test'
test = test[['full_path', 'Image Index', 'fold', 'Atelectasis', 'Cardiomegaly', 'Effusion', 'Infiltration',
             'Mass', 'Nodule', 'Pneumonia', 'Pneumothorax']]
train.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)

In [ ]:
import torch
import torch.nn as nn
import torchvision

from torch.utils.data import (
    Dataset,
    DataLoader,
)

from torchvision.datasets import ImageFolder
from torchvision.transforms import (
    RandomResizedCrop,
    RandomHorizontalFlip,
    ColorJitter,
    RandomGrayscale,
    RandomApply,
    Compose,
    GaussianBlur,
    ToTensor,
)
import torchvision.models as models

import os
import glob
import time
from skimage import io
import matplotlib.pyplot as plt

In [ ]:
import torch
import torchvision
from torch import nn

from lightly.loss import BarlowTwinsLoss
from lightly.models.modules import BarlowTwinsProjectionHead
# from lightly.transforms.byol_transform import (
#     BYOLTransform,
#     BYOLView1Transform,
#     BYOLView2Transform,
# )

In [ ]:
from PIL import Image as img

class CreateDataset(Dataset):

    def __init__(self, train, fold, transform):
        self.df = train
        self.transform = transform
        self.df = self.df[self.df['fold'] == fold]

        self.df = self.df.set_index("Image Index")
        self.PRED_LABEL = ['Atelectasis',
                           'Cardiomegaly',
                           'Effusion',
                           'Infiltration',
                           'Mass',
                           'Nodule',
                           'Pneumonia',
                           'Pneumothorax']
        RESULT_PATH = "results/"

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        image = img.open(self.df.iloc[idx, 0])
        image = image.convert('RGB')

        label = np.zeros(len(self.PRED_LABEL), dtype=int)
        for i in range(0, len(self.PRED_LABEL)):
            if(self.df[self.PRED_LABEL[i].strip()].iloc[idx].astype('int') > 0):
                label[i] = self.df[self.PRED_LABEL[i].strip()
                                  ].iloc[idx].astype('int')

        if self.transform:
            image = self.transform(image)

        return image

In [ ]:
from typing import Optional, Tuple, Union

import torchvision.transforms as T
from PIL.Image import Image
from torch import Tensor

from lightly.transforms.gaussian_blur import GaussianBlur
from lightly.transforms.multi_view_transform import MultiViewTransform
from lightly.transforms.rotation import random_rotation_transform
from lightly.transforms.solarize import RandomSolarization
from lightly.transforms.utils import IMAGENET_NORMALIZE


class BYOLView1Transform:
    def __init__(
        self,
        input_size: int = 224,
        cj_prob: float = 0.8,
        cj_strength: float = 1.0,
        cj_bright: float = 0.4,
        cj_contrast: float = 0.4,
        cj_sat: float = 0.2,
        cj_hue: float = 0.1,
        min_scale: float = 0.08,
        random_gray_scale: float = 0.2,
        gaussian_blur: float = 1.0,
        solarization_prob: float = 0.0,
        kernel_size: Optional[float] = None,
        sigmas: Tuple[float, float] = (0.1, 2),
        vf_prob: float = 0.0,
        hf_prob: float = 1.0,
        rr_prob: float = 1.0,
        rr_degrees: Union[None, float, Tuple[float, float]] = 10,
        normalize: Union[None, dict] = IMAGENET_NORMALIZE,
    ):
        color_jitter = T.ColorJitter(
            brightness=cj_strength * cj_bright,
            contrast=cj_strength * cj_contrast,
            saturation=cj_strength * cj_sat,
            hue=cj_strength * cj_hue,
        )

        transform = [
            # T.RandomResizedCrop(size=input_size, scale=(min_scale, 1.0)),
            T.Resize(224),
            random_rotation_transform(rr_prob=rr_prob, rr_degrees=rr_degrees),
            T.RandomHorizontalFlip(p=hf_prob),
            # T.RandomVerticalFlip(p=vf_prob),
            # T.RandomApply([color_jitter], p=cj_prob),
            # T.RandomGrayscale(p=random_gray_scale),
            # GaussianBlur(kernel_size=kernel_size, sigmas=sigmas, prob=gaussian_blur),
            # RandomSolarization(prob=solarization_prob),
            T.ToTensor(),
        ]
        if normalize:
            transform += [T.Normalize(mean=normalize["mean"], std=normalize["std"])]
        self.transform = T.Compose(transform)

    def __call__(self, image: Union[Tensor, Image]) -> Tensor:
        """
        Applies the transforms to the input image.

        Args:
            image:
                The input image to apply the transforms to.

        Returns:
            The transformed image.

        """
        return self.transform(image)


class BYOLView2Transform:
    def __init__(
        self,
        input_size: int = 224,
        cj_prob: float = 0.8,
        cj_strength: float = 1.0,
        cj_bright: float = 0.4,
        cj_contrast: float = 0.4,
        cj_sat: float = 0.2,
        cj_hue: float = 0.1,
        min_scale: float = 0.08,
        random_gray_scale: float = 0.2,
        gaussian_blur: float = 0.1,
        solarization_prob: float = 0.2,
        kernel_size: Optional[float] = None,
        sigmas: Tuple[float, float] = (0.1, 2),
        vf_prob: float = 0.0,
        hf_prob: float = 1.0,
        rr_prob: float = 1.0,
        rr_degrees: Union[None, float, Tuple[float, float]] = 10,
        normalize: Union[None, dict] = IMAGENET_NORMALIZE,
    ):
        color_jitter = T.ColorJitter(
            brightness=cj_strength * cj_bright,
            contrast=cj_strength * cj_contrast,
            saturation=cj_strength * cj_sat,
            hue=cj_strength * cj_hue,
        )

        transform = [
            # T.RandomResizedCrop(size=input_size, scale=(min_scale, 1.0)),
            T.Resize(224),
            random_rotation_transform(rr_prob=rr_prob, rr_degrees=rr_degrees),
            T.RandomHorizontalFlip(p=hf_prob),
            # T.RandomVerticalFlip(p=vf_prob),
            # T.RandomApply([color_jitter], p=cj_prob),
            # T.RandomGrayscale(p=random_gray_scale),
            # GaussianBlur(kernel_size=kernel_size, sigmas=sigmas, prob=gaussian_blur),
            # RandomSolarization(prob=solarization_prob),
            T.ToTensor(),
        ]
        if normalize:
            transform += [T.Normalize(mean=normalize["mean"], std=normalize["std"])]
        self.transform = T.Compose(transform)

    def __call__(self, image: Union[Tensor, Image]) -> Tensor:
        """
        Applies the transforms to the input image.

        Args:
            image:
                The input image to apply the transforms to.

        Returns:
            The transformed image.

        """
        return self.transform(image)


class BYOLTransform(MultiViewTransform):
    """Implements the transformations for BYOL[0].

    Input to this transform:
        PIL Image or Tensor.

    Output of this transform:
        List of Tensor of length 2.

    Applies the following augmentations by default:
        - Random resized crop
        - Random horizontal flip
        - Color jitter
        - Random gray scale
        - Gaussian blur
        - Solarization
        - ImageNet normalization

    Note that SimCLR v1 and v2 use similar augmentations. In detail, BYOL has
    asymmetric gaussian blur and solarization. Furthermore, BYOL has weaker
    color jitter compared to SimCLR.

    - [0]: Bootstrap Your Own Latent, 2020, https://arxiv.org/pdf/2006.07733.pdf

    Input to this transform:
        PIL Image or Tensor.

    Output of this transform:
        List of [tensor, tensor].

    Attributes:
        view_1_transform: The transform for the first view.
        view_2_transform: The transform for the second view.
    """

    def __init__(
        self,
        view_1_transform: Union[BYOLView1Transform, None] = None,
        view_2_transform: Union[BYOLView2Transform, None] = None,
    ):
        # We need to initialize the transforms here
        view_1_transform = view_1_transform or BYOLView1Transform()
        view_2_transform = view_2_transform or BYOLView2Transform()
        super().__init__(transforms=[view_1_transform, view_2_transform])

In [ ]:
transform = BYOLTransform(
    view_1_transform=BYOLView1Transform(),
    view_2_transform=BYOLView2Transform(),
)

dataset = CreateDataset(train, 'train', transform=transform)

dataloader = torch.utils.data.DataLoader(
    dataset,
    batch_size=64,
    shuffle=True,
    drop_last=True,
    num_workers=8,
    pin_memory=True
)

In [ ]:
class BarlowTwins(nn.Module):
    def __init__(self, backbone):
        super().__init__()
        self.backbone = backbone
        self.projection_head = BarlowTwinsProjectionHead(2048, 2048, 2048)

    def forward(self, x):
        x = self.backbone(x).flatten(start_dim=1)
        z = self.projection_head(x)
        return z

In [ ]:
backbone = torchvision.models.resnet50()
# backbone = nn.DataParallel(backbone)
checkpoint = torch.load('/content/drive/MyDrive/barlow/resnet50.pth')
backbone.load_state_dict(checkpoint, strict=False)
backbone = nn.Sequential(*list(backbone.children())[:-1])
backbone

Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)


In [ ]:
model = BarlowTwins(backbone)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
print('Model ready for training')

Model ready for training


In [ ]:
criterion = BarlowTwinsLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.06, momentum=0.9, weight_decay=0.0001)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, 20)
loss_val_list = []

print("Starting Training")
for epoch in range(20):
    t0 = time.time()
    total_loss = 0
    for batch in dataloader:
        x0, x1 = batch
        x0 = x0.to(device)
        x1 = x1.to(device)
        z0 = model(x0)
        z1 = model(x1)
        loss = criterion(z0, z1)
        total_loss += loss.detach()
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    scheduler.step()
    avg_loss = total_loss / len(dataloader)

    if ((epoch+1) == 1):
      model_save_name = f'barlow_chestxray_{epoch+1}_withprojectionhead.pth'
      path = f'/content/drive/MyDrive/barlow/{model_save_name}'
      torch.save(model.state_dict(), path)

    if ((epoch+1) % 4 == 0):
      model_save_name = f'barlow_chestxray_{epoch+1}_withprojectionhead.pth'
      path = f'/content/drive/MyDrive/barlow/{model_save_name}'
      torch.save(model.state_dict(), path)

    if ((epoch+1) % 2 == 0):
      torch.save(model.state_dict(), f'pretrained_barlow_{epoch+1}_withprojectionhead.pth')
      state_dict = {'resnet50_parameters': model.backbone.state_dict()}
      torch.save(state_dict, f'pretrained_barlow_{epoch+1}_backbonemodel.pth')
    print(f"epoch: {epoch+1:>02}, loss: {avg_loss:.5f}. Time taken: {((time.time()-t0)/60):.3f} mins")
    loss_val_list.append(avg_loss)

Starting Training
epoch: 01, loss: 1743.82483. Time taken: 5.892 mins
epoch: 02, loss: 1201.60242. Time taken: 5.615 mins
epoch: 03, loss: 907.69995. Time taken: 5.616 mins
epoch: 04, loss: 678.92389. Time taken: 5.602 mins
epoch: 05, loss: 558.17590. Time taken: 5.616 mins
epoch: 06, loss: 494.75928. Time taken: 5.592 mins
epoch: 07, loss: 459.59607. Time taken: 5.572 mins
epoch: 08, loss: 435.94852. Time taken: 5.601 mins
epoch: 09, loss: 420.47437. Time taken: 5.594 mins
epoch: 10, loss: 409.25507. Time taken: 5.614 mins
epoch: 11, loss: 401.81180. Time taken: 5.559 mins
epoch: 12, loss: 394.55441. Time taken: 5.617 mins
epoch: 13, loss: 390.39508. Time taken: 5.612 mins
epoch: 14, loss: 386.07202. Time taken: 5.614 mins
epoch: 15, loss: 382.88803. Time taken: 5.585 mins
epoch: 16, loss: 380.40851. Time taken: 5.624 mins
epoch: 17, loss: 378.49451. Time taken: 5.597 mins
epoch: 18, loss: 377.20786. Time taken: 5.568 mins
epoch: 19, loss: 376.22583. Time taken: 5.588 mins
epoch: 20, 

In [ ]:
for i in range(len(loss_val_list)):
  loss_val_list[i] = loss_val_list[i].item()
np.save('/content/drive/MyDrive/barlow/barlow_0.06_loss_20.npy', loss_val_list)